<a href="https://colab.research.google.com/github/DhaneshGavimath/NLP-Projects/blob/main/model/chatbot%20modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install jupyterthemes
# !jt -l

In [2]:
# Choosing the notebook theme
# !jt -t monokai

Imports

In [3]:
import pandas as pd
import numpy as np
import json
import nltk 
import sklearn

In [4]:
import tensorflow as tf


In [5]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation

In [6]:
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
"""
classes
"""
intents_classes = ["greetings", "goodbye","fee_structure", "campus_placement", "departments", 
                    "adress","activities", "commute","gratitude"]
intents_classes.sort()
classes_code_label = dict(map(lambda x:(intents_classes.index(x),x),intents_classes))
classes_label_code = dict(map(lambda x:(x,intents_classes.index(x)),intents_classes))
# print("-------------classes_str_to_num-----------\n",classes_str_to_num)
# print("-------------classes_num_to_str-----------\n",classes_num_to_str)

Createing onehot vector representation for output vector

In [8]:
def vector(output_class):
    vec = np.zeros(len(intents_classes),dtype=np.float32)
    vec[intents_classes.index(output_class)] = 1.0
    return vec

In [9]:
classes_vec_label = dict()

for intent in intents_classes:
  classes_vec_label[intent] = vector(intent)

print(classes_vec_label)

{'activities': array([1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), 'adress': array([0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), 'campus_placement': array([0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32), 'commute': array([0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32), 'departments': array([0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32), 'fee_structure': array([0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32), 'goodbye': array([0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32), 'gratitude': array([0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32), 'greetings': array([0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)}


### Load Data
 Here we will be loading the json we made containing the questions user might ask.
 as we run this file it show load the training file, perform datacleaning, vector repressentation
 and finally creating a dataframe and write to current folder. 
 we will be running this file only to retrain the model with updated data we might get new words.
    

In [10]:
def load_chat_data():
    # loading the data  
    with open("train_classes.json",'r') as jdata:
        data = json.load(jdata)

    train_data = []

    for intent, sentences in data.items():
        input_target_pair = list(map(lambda x: (x,intent),sentences))
        train_data.extend(input_target_pair)

    train_dataframe = pd.DataFrame(train_data, columns = ["input","target"])
    return train_dataframe

data = load_chat_data()
data.head()

,input,target
0,Hi,greetings
1,Hello,greetings
2,Good Morning!,greetings
3,Good Evening!,greetings
4,bye,goodbye


In [11]:
"""
Replacing the target with numeric value
"""
data["target"] = data["target"].map(classes_vec_label)
# data["target"] = data["target"].map(classes_label_code)

In [12]:
data.head()

,input,target
0,Hi,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
1,Hello,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
2,Good Morning!,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
3,Good Evening!,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
4,bye,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"


In [13]:
def tokenize(data):
    # Word tokenize
    tokens = word_tokenize(data)
    return tokens

def remove_junks(tokens):
    # Removing punctuations
    wo_punctuations  = set(tokens).difference(set(punctuation))
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    wo_stopwords_puncts = wo_punctuations.difference(stop_words)
    return list(wo_stopwords_puncts)

def stemming(clean_words):
    stem_obj = PorterStemmer()
    lower_case_words = list(map(lambda x:x.lower(),clean_words))
    stem_words = list(map(lambda x:stem_obj.stem(x),lower_case_words))
    return stem_words

def create_vocabulary(data, column):
    # creating a vocabulary out of text column
    vocab = []
    for record in data[column].values:
      vocab.extend(record)
    return set(vocab)

def clean_data(df, column):
    df[column]= df[column].apply(tokenize)
    df[column]= df[column].apply(remove_junks)
    df[column]= df[column].apply(stemming)
    return df

In [14]:
data = clean_data(data,"input")


In [15]:
vocabulary = create_vocabulary(data, "input")
vocab_dict = {"vocabulary": list(vocabulary)}

In [16]:
# Writing Vocabulary to json file
with open("vocabulary.json","w") as voc_save_file:
  json.dump(vocab_dict,voc_save_file)

In [17]:
# Let's represent each text with bag-of-words method
def bag_of_words_representation(df, column):
    vocab = vocab_dict["vocabulary"]
    def vector_represent(record):
      bag_of_words = np.zeros(len(vocabulary))
      for word in record:
        if word in vocab:
          bag_of_words[vocab.index(word)] += 1
      return bag_of_words
      
    vector_matrix = list(map(lambda x: vector_represent(x),df[column].values))
    bow_df = pd.DataFrame(vector_matrix,columns=vocabulary)
    df.drop(columns=[column], inplace=True)
    final_df = pd.concat([df,bow_df],axis=1)
    return final_df
  


In [18]:
data = bag_of_words_representation(data,"input")

In [19]:
# data.head()

In [20]:
y_data =pd.DataFrame(data["target"])
x_data = data.drop(columns=['target'])
print(x_data.head())
print("****************************")
print(y_data)


   commut  great  travel  work  appreci  structur  sport  avail  adress  \
0     0.0    0.0     0.0   0.0      0.0       0.0    0.0    0.0     0.0   
1     0.0    0.0     0.0   0.0      0.0       0.0    0.0    0.0     0.0   
2     0.0    0.0     0.0   0.0      0.0       0.0    0.0    0.0     0.0   
3     0.0    0.0     0.0   0.0      0.0       0.0    0.0    0.0     0.0   
4     0.0    0.0     0.0   0.0      0.0       0.0    0.0    0.0     0.0   

   provid  ...  subject  valu  hello  job  station  night  nearbi  cours  fee  \
0     0.0  ...      0.0   0.0    0.0  0.0      0.0    0.0     0.0    0.0  0.0   
1     0.0  ...      0.0   0.0    1.0  0.0      0.0    0.0     0.0    0.0  0.0   
2     0.0  ...      0.0   0.0    0.0  0.0      0.0    0.0     0.0    0.0  0.0   
3     0.0  ...      0.0   0.0    0.0  0.0      0.0    0.0     0.0    0.0  0.0   
4     0.0  ...      0.0   0.0    0.0  0.0      0.0    0.0     0.0    0.0  0.0   

   taxi  
0   0.0  
1   0.0  
2   0.0  
3   0.0  
4   0.0  

[

Converting Numpy inputs to tensors


In [40]:
x_tensors = tf.convert_to_tensor(x_data)

y_df = pd.DataFrame(dict(zip(y_data.index,y_data['target'].values))).T
y_df.columns = intents_classes

# y_tensors = tf.constant(y_df.values, dtype = tf.float32,shape=[35,9])
y_tensors = tf.convert_to_tensor(y_df)

In [41]:
x_tensors

<tf.Tensor: shape=(35, 45), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])>

In [42]:
y_tensors

<tf.Tensor: shape=(35, 9), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
      


**Model**



In [43]:
input_layer = tf.keras.Input(shape=(x_data.shape[1],))
middle_layer = tf.keras.layers.Dense(32, activation='relu') 
output_layer =  tf.keras.layers.Dense(9) 

model = tf.keras.Sequential()
model.add(input_layer)
model.add(middle_layer)
model.add(output_layer)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                1472      
                                                                 
 dense_5 (Dense)             (None, 9)                 297       
                                                                 
Total params: 1,769
Trainable params: 1,769
Non-trainable params: 0
_________________________________________________________________


In [44]:
optim = tf.keras.optimizers.SGD(learning_rate=0.01)

In [45]:
model.compile(optimizer=optim,
              loss=tf.keras.losses.CategoricalCrossentropy())


In [46]:
history = model.fit(x_tensors, y_tensors, epochs=1000)

Epoch 1/1000
2/2 [==============================] - 1s 11ms/step - loss: 7.1456
Epoch 2/1000
2/2 [==============================] - 0s 8ms/step - loss: 6.2345
Epoch 3/1000
2/2 [==============================] - 0s 7ms/step - loss: 7.0298
Epoch 4/1000
2/2 [==============================] - 0s 8ms/step - loss: 6.4250
Epoch 5/1000
2/2 [==============================] - 0s 8ms/step - loss: 7.1982
Epoch 6/1000
2/2 [==============================] - 0s 11ms/step - loss: 7.3173
Epoch 7/1000
2/2 [==============================] - 0s 5ms/step - loss: 7.6211
Epoch 8/1000
2/2 [==============================] - 0s 9ms/step - loss: 7.2028
Epoch 9/1000
2/2 [==============================] - 0s 10ms/step - loss: 6.2594
Epoch 10/1000
2/2 [==============================] - 0s 6ms/step - loss: 6.6697
Epoch 11/1000
2/2 [==============================] - 0s 5ms/step - loss: 7.1214
Epoch 12/1000
2/2 [==============================] - 0s 5ms/step - loss: 7.1072
Epoch 13/1000
2/2 [===========================

In [28]:
help(history)

Help on History in module keras.callbacks object:

class History(Callback)
 |  Callback that records events into a `History` object.
 |  
 |  This callback is automatically applied to
 |  every Keras model. The `History` object
 |  gets returned by the `fit` method of models.
 |  
 |  Example:
 |  
 |  >>> model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])
 |  >>> model.compile(tf.keras.optimizers.SGD(), loss='mse')
 |  >>> history = model.fit(np.arange(100).reshape(5, 20), np.zeros(5),
 |  ...                     epochs=10, verbose=1)
 |  >>> print(history.params)
 |  {'verbose': 1, 'epochs': 10, 'steps': 1}
 |  >>> # check the keys of history object
 |  >>> print(history.history.keys())
 |  dict_keys(['loss'])
 |  
 |  Method resolution order:
 |      History
 |      Callback
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  on_epoch_end(self, epoch, logs=None)
 |